In [1]:
import pandas as pd
import numpy as np
#from numpy import random
import random
from scipy import stats as st

In [18]:
#loading in NLP values
df1 = pd.read_csv('wikipages_nlp.csv', index_col=0)
df1.head(10)

,Name,Year,inTransWomen,inTransMen,inNB,inTrans,inWomen,inMen,inLGBT,inBisexual_people,...,empath_weapon,empath_children,empath_monster,empath_ocean,empath_giving,empath_contentment,empath_writing,empath_rural,empath_positive_emotion,empath_musical
0,Kjell Aamot,1950,False,False,False,False,False,False,False,False,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.006897,0.000000,0.000000,0.000000
1,Olav Aaraas,1950,False,False,False,False,False,False,False,False,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.006211,0.000000,0.000000,0.000000
2,Marc Aaronson,1950,False,False,False,False,False,False,False,False,...,0.000000,0.000000,0.0,0.001025,0.000000,0.0,0.000000,0.000000,0.001025,0.000000
3,Ülle Aaskivi,1950,False,False,False,False,True,False,False,False,...,0.000000,0.002304,0.0,0.000000,0.002304,0.0,0.000000,0.000000,0.002304,0.000000
4,Oğuz Abadan,1950,False,False,False,False,False,False,False,False,...,0.000000,0.003942,0.0,0.000000,0.001314,0.0,0.000000,0.000000,0.000000,0.024967
5,Ahmet Abakay,1950,False,False,False,False,False,True,False,False,...,0.000000,0.015075,0.0,0.000000,0.000000,0.0,0.012563,0.001256,0.003769,0.000000
6,Martine Aballéa,1950,False,False,False,False,True,False,True,False,...,0.000000,0.001067,0.0,0.000000,0.000000,0.0,0.001067,0.000000,0.000000,0.003202
7,Gadzhi Abashilov,1950,False,False,False,False,False,False,False,False,...,0.001996,0.001996,0.0,0.000000,0.000000,0.0,0.003992,0.000000,0.001996,0.000000
8,Kamil Al-Abbasi,1950,False,False,False,False,False,False,False,False,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.008333,0.000000,0.000000,0.000000
9,Aslan Abbasov,1950,False,False,False,False,False,False,False,False,...,0.000000,0.000000,0.0,0.000000,0.003953,0.0,0.035573,0.000000,0.000000,0.000000


In [19]:
#dropping extra columns
df1=df1.drop(df1.columns[2:13],axis=1)#df1.columns[2:13]
#df1.head()

In [20]:
colNames=df1.columns[6:]

In [4]:
#reading in wikidata data
df2 = pd.read_csv('wikidataSupportData.csv',keep_default_na=False)
#df2.head()

In [56]:
df2.to_csv('wikidataSupportData.csv',index=False)

In [5]:
#cleaning up wikidata data by dropping rows that have a broken wikidata link in either the P21 or P91 columns

df=df2[~df2.P21.str.contains('|'.join(['http://www.wikidata.org/']))]
df=df[~df.P91.str.contains('|'.join(['http://www.wikidata.org/']))]
df['P91'].unique()

array(['', 'lesbianism', 'non-heterosexuality', 'asexuality', 'gay',
       'homosexuality', 'heterosexuality', 'bisexuality', 'pansexuality',
       'hasbian', 'homoeroticism', 'sapiosexuality', 'male homosexuality',
       'demisexuality', 'homoromantic', 'sexual fluidity', 'aromanticism',
       'gray asexuality', 'bisexual', 'gay men'], dtype=object)

In [6]:
#seperating into gay/not gay sexuality
ng=pd.concat([df[df['P91']==''],df[df['P91']=='heterosexuality']])
g=df[~df.QID.isin(ng.QID)]

In [7]:
ng.head()

,QID,Name,P21,P21ref,P91,P91ref,P6553,P6553ref
0,Q66764,Marion Becker,female,,,,,
1,Q69588,Yuri N Maltsev,male,,,,,
2,Q70195,Klaus Regling,male,,,,,
3,Q74800,Hans-Joachim Zillmer,male,,,,,
4,Q75733,Gisbert Haefs,male,,,,,


In [8]:
g.head()

,QID,Name,P21,P21ref,P91,P91ref,P6553,P6553ref
5043,Q6794690,Max Dashu,female,,lesbianism,,,
5469,Q7280044,Radclyffe,female,,lesbianism,,,
9181,Q16543603,Carmen Muñoz,female,,lesbianism,,,
11473,Q21567781,Tamás Király,male,,non-heterosexuality,,,
11593,Q22004198,Beth Elliott,transgender female,,lesbianism,,,


In [9]:
#splitting into cis/non-cis gender ID
cis=pd.concat([df[df['P21']==''],
             df[df['P21']=='female'],
             df[df['P21']=='cisgender male'],
             df[df['P21']=='cisgender female'],
             df[df['P21']=='male']])
t=df[~df.QID.isin(cis.QID)]

In [10]:
t['P21'].unique()

array(['transgender female', 'non-binary', 'genderqueer',
       'transgender male', 'intersex', 'kathoey', 'two-spirit',
       'genderfluid', 'transgender person', 'third gender', 'neutral sex',
       'travesti', 'bigender', 'female organism', 'pangender', 'agender',
       'queer', 'male organism', 'muxe', 'intersex person',
       'transmasculine', 'transfeminine', 'demiboy', 'neutrois',
       'assigned female at birth', 'nigger'], dtype=object)

In [11]:
cis['P21'].unique()

array(['', 'female', 'cisgender male', 'cisgender female', 'male'],
      dtype=object)

In [12]:
#using above to divide queer/non-queer
q=pd.merge(t,g, how='outer').drop_duplicates(subset=['QID'])
#q.tail()

In [13]:
nq=df[~df.QID.isin(q.QID)]
nq.head()

,QID,Name,P21,P21ref,P91,P91ref,P6553,P6553ref
0,Q66764,Marion Becker,female,,,,,
1,Q69588,Yuri N Maltsev,male,,,,,
2,Q70195,Klaus Regling,male,,,,,
3,Q74800,Hans-Joachim Zillmer,male,,,,,
4,Q75733,Gisbert Haefs,male,,,,,


In [14]:
nq.tail()

,QID,Name,P21,P21ref,P91,P91ref,P6553,P6553ref
2126571,Q102252979,Eydan Yusoff bin Daniel James Tyler,male,,,,,
2126572,Q105823098,Aksara Asa Nasution,male,,,,,
2126573,Q107742532,Luke Neto,male,,,,,
2126574,Q109664709,Maria Alexandra de Roumanie,female,,,,,
2126575,Q104153420,Prithvi Ambani,male,,,,,


In [15]:
#randomly sample rows for statistical tests
print(len(g),len(ng),len(t),len(cis),len(q),len(nq))

2461 2124073 1627 2124821 3649 2122588


In [17]:
gIndex=df1.index[df1['QID'].isin(g['QID'])].tolist()
ngIndex=df1.index[df1['QID'].isin(ng['QID'])].tolist()
tIndex=df1.index[df1['QID'].isin(t['QID'])].tolist()
cisIndex=df1.index[df1['QID'].isin(cis['QID'])].tolist()
qIndex=df1.index[df1['QID'].isin(q['QID'])].tolist()
nqIndex=df1.index[df1['QID'].isin(nq['QID'])].tolist()

In [47]:
indices=pd.DataFrame()
indices['g']=gSample
indices['ng']=ngSample
indices['t']=tSample
indices['c']=cisSample
indices['q']=qSample
indices['nq']=nqSample
indices.tail()

,g,ng,t,c,q,nq
495,569806,569962,522502,102031,269880,518394
496,462508,107552,568905,412757,293731,451701
497,99784,111656,222065,366925,36721,386772
498,667396,545300,624304,549364,417848,383384
499,690553,411772,547477,668350,204273,589763


In [48]:
indices.to_csv('nlpSampledIndices.csv',index=False)

In [18]:
print(len(gIndex),len(tIndex),len(qIndex))

1695 733 2360


In [19]:
#gSample=g.sample(n=1000)
#gSample.head()
#ngSample=ng.sample(n=1000)
#tSample=t.sample(n=1000)
#cisSample=cis.sample(n=1000)
#qSample=q.sample(n=1000)
#nqSample=nq.sample(n=1000)

In [20]:
gSample=random.sample(gIndex,500)
ngSample=random.sample(ngIndex,500)
tSample=random.sample(tIndex,500)
cisSample=random.sample(cisIndex,500)
qSample=random.sample(qIndex,500)
nqSample=random.sample(nqIndex,500)
#print(gSample,len(gSample))

In [27]:
results=pd.DataFrame(columns=['metric','orientationStatistic','orientationPValue','genderStatistic','genderPValue','queernessStatistic','queernessPValue'])

In [28]:
for col in colNames:
    a=df1[col].iloc[gSample]
    b=df1[col].iloc[ngSample]
    c=df1[col].iloc[tSample]
    d=df1[col].iloc[cisSample]
    e=df1[col].iloc[qSample]
    f=df1[col].iloc[nqSample]
    x,y=st.ttest_ind(a,b)
    z,w=st.ttest_ind(c,d)
    s,t=st.ttest_ind(e,f)
    results.loc[len(results.index)] = [col,x,y,z,w,s,t]
results.head()    

,metric,orientationStatistic,orientationPValue,genderStatistic,genderPValue,queernessStatistic,queernessPValue
0,word_count,12.266117,2.567733e-32,8.657466,1.910704e-17,12.053023,2.489879e-31
1,vader_compound,0.189298,8.498980e-01,-2.669277,7.724869e-03,-0.367839,7.130713e-01
2,vader_neg,7.395141,2.988532e-13,9.983254,1.952613e-22,9.474023,1.887993e-20
3,vader_neu,-9.753568,1.572129e-21,-8.090795,1.711335e-15,-9.847690,6.719267e-22
4,vader_pos,6.480608,1.433038e-10,2.637032,8.493461e-03,5.804342,8.674733e-09


In [36]:
df1['word_count'].iloc[gSample].describe()

count      500.000000
mean      3176.946000
std       3469.520408
min        118.000000
25%       1045.750000
50%       2006.000000
75%       3869.000000
max      25651.000000
Name: word_count, dtype: float64

In [37]:
df1['word_count'].iloc[ngSample].describe()

count      500.000000
mean      1132.988000
std       1358.658585
min         68.000000
25%        369.250000
50%        737.000000
75%       1340.000000
max      11911.000000
Name: word_count, dtype: float64

In [42]:
results.to_csv('nlpStatsResults.csv', index=False)

In [40]:
#results['orientationSignificant']='no'
#results['genderSignificant']='no'
#results['queernessSignificant']='no'

oSig=[]
gSig=[]
qSig=[]


for index,row in results.iterrows():
    if row['orientationPValue'] < 0.00025:
        oSig.append('yes')
    else:
        oSig.append('no')
    if row['genderPValue'] < 0.00025:
        gSig.append('yes')
    else:
        gSig.append('no')
    if row['queernessPValue'] < 0.00025:
        qSig.append('yes')
    else:
        qSig.append('no')
    
results['orientationSignificant']=oSig
results['genderSignificant']=gSig
results['queernessSignificant']=qSig
results.head()

,metric,orientationStatistic,orientationPValue,genderStatistic,genderPValue,queernessStatistic,queernessPValue,orientationSignificant,genderSignificant,queernessSignificant
0,word_count,12.266117,2.567733e-32,8.657466,1.910704e-17,12.053023,2.489879e-31,yes,yes,yes
1,vader_compound,0.189298,8.498980e-01,-2.669277,7.724869e-03,-0.367839,7.130713e-01,no,no,no
2,vader_neg,7.395141,2.988532e-13,9.983254,1.952613e-22,9.474023,1.887993e-20,yes,yes,yes
3,vader_neu,-9.753568,1.572129e-21,-8.090795,1.711335e-15,-9.847690,6.719267e-22,yes,yes,yes
4,vader_pos,6.480608,1.433038e-10,2.637032,8.493461e-03,5.804342,8.674733e-09,yes,no,yes


In [38]:
results.tail()

,metric,orientationStatistic,orientationPValue,genderStatistic,genderPValue,queernessStatistic,queernessPValue,orientationSignificant,genderSignificant,queernessSignificant
194,empath_contentment,2.983801,2.916221e-03,5.933493,4.086046e-09,3.694436,2.323427e-04,yes,yes,yes
195,empath_writing,-5.220278,2.173354e-07,-4.601511,4.734156e-06,-5.745718,1.214978e-08,yes,yes,yes
196,empath_rural,-0.616231,5.378828e-01,0.902341,3.670936e-01,2.216270,2.689788e-02,no,no,yes
197,empath_positive_emotion,4.813353,1.713565e-06,6.435607,1.906277e-10,8.573415,3.782236e-17,yes,yes,yes
198,empath_musical,3.762200,1.782329e-04,3.763675,1.771995e-04,6.009071,2.612356e-09,yes,yes,yes


In [34]:
a.describe()

count    500.000000
mean       0.004992
std        0.006371
min        0.000000
25%        0.000520
50%        0.002492
75%        0.007005
max        0.039537
Name: empath_musical, dtype: float64

In [ ]:
nlpG=df1[~df1.QID.isin(gSample['QID'].unique())]
nlpG.head()

In [ ]:
nlpNG=df1[~df1.QID.isin(ngSample['QID'].unique())]
nlpNG.head()

In [ ]:
nlpT=df1[~df1.QID.isin(tSample['QID'].unique())]
nlpT.head()

In [ ]:
nlpCis=df1[~df1.QID.isin(cisSample['QID'].unique())]
nlpCis.head()

In [ ]:
nlpQ=df1[~df1.QID.isin(qSample['QID'].unique())]
nlpQ.head()

In [ ]:
nlpNQ=df1[~df1.QID.isin(nqSample['QID'].unique())]
nlpNQ.head()

# Slicing by Occupation/Field

In [2]:
df3=pd.read_csv('wikidataSupportDataOccupation.csv',keep_default_na=False)
df3.head()

,QID,Name,P21,P21ref,P91,P91ref,P6553,P6553ref,Occupation,Field,Position
0,Q671878,Noriyuki Konishi,male,,,,,,mangaka,,
1,Q672539,Bob Mansfield,male,,,,,,engineer,,
2,Q672539,Bob Mansfield,male,,,,,,computer scientist,,
3,Q674270,Hans Böhm,male,,,,,,television presenter,,
4,Q674270,Hans Böhm,male,,,,,,journalist,,


In [10]:
actorsOccupationSynonyms=['actor','film actor','television actor']
activistsSynonyms=['activist','political activist','LGBTI rights activist']
businessSynonyms=['businessperson','entrepreneur']  

In [13]:
dfa=df3[df3['Occupation'].isin(actorsOccupationSynonyms)]
dfa=dfa.drop_duplicates(subset=['QID'])
dfa.head()

,QID,Name,P21,P21ref,P91,P91ref,P6553,P6553ref,Occupation,Field,Position
13,Q710261,Judy Ongg,female,,,,,,actor,,
59,Q729902,David Lloyd,male,,,,,,actor,,
113,Q522024,Cristina Galbó,female,,,,,,film actor,,
120,Q525109,Ronny Yu,male,,,,,,actor,,
177,Q544245,Safy Boutella,male,,,,,,actor,,


In [15]:
dfb=df3[df3['Occupation'].isin(activistsSynonyms)]
dfb=dfb.drop_duplicates(subset=['QID'])
dfb.head()

,QID,Name,P21,P21ref,P91,P91ref,P6553,P6553ref,Occupation,Field,Position
280,Q633404,Robert Arricau,male,,,,,,political activist,,
1885,Q3172400,Jean Groix,male,,,,,,political activist,,
2560,Q3296464,María Poumier,female,,,,,,political activist,,
2655,Q4491960,Matt Foreman,male,,,,,,LGBTI rights activist,,
5353,Q4933390,Bob Mellors,male,,,,,,LGBTI rights activist,,


In [16]:
dfc=df3[df3['Occupation'].isin(businessSynonyms)]
dfc=dfc.drop_duplicates(subset=['QID'])
dfc.head()

,QID,Name,P21,P21ref,P91,P91ref,P6553,P6553ref,Occupation,Field,Position
53,Q723515,Chris Taylor,male,,,,,,entrepreneur,,
95,Q746053,Jay Rasulo,male,,,,,,businessperson,,
154,Q539624,El-Sayyid el-Badawi,male,,,,,,businessperson,,
213,Q578385,Kochouseph Chittilappilly,male,,,,,,businessperson,,
318,Q780081,Yazid Sabeg,male,,,,,,businessperson,,UNESCO Goodwill Ambassador


In [17]:
print(len(dfa),len(dfb),len(dfc))

147284 3852 33022


In [22]:
datas=[dfa,dfb,dfc]
labels=['actors','activists','business']
for i in [0,1,2]:
    df=datas[i]
    df=df[~df.P21.str.contains('|'.join(['http://www.wikidata.org/']))]
    df=df[~df.P91.str.contains('|'.join(['http://www.wikidata.org/']))]
    
    ng=pd.concat([df[df['P91']==''],df[df['P91']=='heterosexuality']])
    g=df[~df.QID.isin(ng.QID)]
    cis=pd.concat([df[df['P21']==''],
                   df[df['P21']=='female'],
                   df[df['P21']=='cisgender male'],
                   df[df['P21']=='cisgender female'],
                   df[df['P21']=='male']])
    t=df[~df.QID.isin(cis.QID)]
    q=pd.merge(t,g, how='outer').drop_duplicates(subset=['QID'])
    nq=df[~df.QID.isin(q.QID)]
    sampSize=min(len(ng),len(g),len(cis),len(t),len(q),len(nq))
    print(labels[i],sampSize)
    gSample=df1[df1['QID'].isin(random.sample(g['QID'].to_list(),sampSize))]
    ngSample=df1[df1['QID'].isin(random.sample(ng['QID'].to_list(),sampSize))]
    tSample=df1[df1['QID'].isin(random.sample(t['QID'].to_list(),sampSize))]
    cisSample=df1[df1['QID'].isin(random.sample(cis['QID'].to_list(),sampSize))]
    qSample=df1[df1['QID'].isin(random.sample(q['QID'].to_list(),sampSize))]
    nqSample=df1[df1['QID'].isin(random.sample(nq['QID'].to_list(),sampSize))]
    temp=pd.DataFrame(columns=['metric','orientationStatistic','orientationPValue','genderStatistic','genderPValue','queernessStatistic','queernessPValue'])
    for col in colNames:
#         a=df1[col].iloc[gSample]
#         b=df1[col].iloc[ngSample]
#         c=df1[col].iloc[tSample]
#         d=df1[col].iloc[cisSample]
#         e=df1[col].iloc[qSample]
#         f=df1[col].iloc[nqSample]
        x,y=st.ttest_ind(gSample[col],ngSample[col])
        z,w=st.ttest_ind(tSample[col],cisSample[col])
        s,t=st.ttest_ind(qSample[col],nqSample[col])
        temp.loc[len(temp.index)] = [col,x,y,z,w,s,t]
    temp.to_csv('stats_'+labels[i]+'.csv',index=False)

actors 300
activists 133
business 16


In [27]:
results=pd.read_csv('stats_activists.csv',keep_default_na=False)

In [28]:
oSig=[]
gSig=[]
qSig=[]


for index,row in results.iterrows():
    if row['orientationPValue'] < 0.00025:
        oSig.append('yes')
    else:
        oSig.append('no')
    if row['genderPValue'] < 0.00025:
        gSig.append('yes')
    else:
        gSig.append('no')
    if row['queernessPValue'] < 0.00025:
        qSig.append('yes')
    else:
        qSig.append('no')
    
results['orientationSignificant']=oSig
results['genderSignificant']=gSig
results['queernessSignificant']=qSig
results.head()

,metric,orientationStatistic,orientationPValue,genderStatistic,genderPValue,queernessStatistic,queernessPValue,orientationSignificant,genderSignificant,queernessSignificant
0,word_count,2.694395,0.007878,1.620513,0.107280,0.895600,0.372161,no,no,no
1,vader_compound,0.995448,0.321166,2.645170,0.009059,1.140032,0.256419,no,no,no
2,vader_neg,-1.594783,0.112923,-0.978975,0.329212,-1.739497,0.084370,no,no,no
3,vader_neu,-0.045983,0.963387,0.197558,0.843666,0.792226,0.429706,no,no,no
4,vader_pos,1.735582,0.084748,0.850521,0.396428,1.073911,0.284899,no,no,no


In [29]:
results.to_csv('stats_activists.csv',index=False)